## Import relevant libraries and functions

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator, 
    perform_experiments,
    Samplers, 
)
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.analysis import (feature_scoring, pairs_plotting)

from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

## Prepare model and parameters

In [2]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

## Define zero policy

In [3]:
# defining specific policies

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

zeropolicy = Policy("zero policy",**dict(get_do_nothing_dict()))

## Run and save open exploration experiments

In [ ]:
n_exp = 1000

with MultiprocessingEvaluator(dike_model) as evaluator:
    sa_results = evaluator.perform_experiments(scenarios=n_exp, policies=20, uncertainty_sampling=Samplers.SOBOL)

experiments, outcomes = sa_results

experiments.to_pickle('experiments_SOBOL_1000x20')

with open('outcomes_SOBOL_1000x20', 'wb') as f:
    pickle.dump(outcomes, f)

In [ ]:
with open('experiments_SOBOL_1000x20', 'rb') as file:
    experiments_SOBOL_OE_SA = pickle.load(file)
    
with open('outcomes_SOBOL_1000x20', 'rb') as file:
    outcomes_SOBOL_OE_SA = pickle.load(file)

## Analyse open exploration

In [ ]:
fig, axes = pairs_plotting.pairs_scatter(experiments_SOBOL_OE_SA, outcomes_SOBOL_OE_SA, group_by='policy', legend=False)
fig.set_size_inches(12, 12)
plt.show()

## Feature scoring

In [ ]:
fs = feature_scoring.get_feature_scores_all(experiments_SOBOL_OE_SA, outcomes_SOBOL_OE_SA)
sns.heatmap(fs, cmap="viridis", annot=True)
plt.show()

## Analyse SOBOL sensitivity

Eventueel hier twee runs gaan doen: eentje voor alleen uncertainties met zero policy, en een met reference scenario en dan veel policies

In [ ]:
problem = get_SALib_problem(uncertainties)

Si = sobol.analyze(problem, outcomes['xx'], calc_second_order=True, print_to_console=True)